In [1]:
import pymongo
import pprint
import json
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy  as np
import json
import re
import openpyxl
from csv import writer
from openpyxl import load_workbook
from pandas.io.json import json_normalize



# connect to the mongoclient
client = pymongo.MongoClient('mongodb://localhost:27017')

# get the database
database = client['AppDataBase']

Users = pd.DataFrame(list(database["Users"].find()))
cursor = database["Users"].find()
df = json_normalize(cursor)
NumberOfUser=(len(Users))

Hints = pd.DataFrame(list(database["Hints"].find()))

Words = pd.DataFrame(list(database["Words"].find()))

Questions = pd.DataFrame(list(database["Questions"].find()))



    


In [2]:
#Extract Features from Hint File

def Hints_Features(email):
    TempHint=Hints[(Hints["userid"] == email)]
    try:
        TotalHintTime = TempHint['time'].sum()
    except: 
        TotalHintTime=0
    
    try:    
        HintsTextTime=TempHint['time'][TempHint["hinttype"]=="text"].sum()
    except: 
        HintsTextTime=0
    
    try:    
        HintsImageTime=TempHint['time'][TempHint["hinttype"]=="image"].sum()
    except:    
        HintsImageTime=0
    
    try:    
        HintsVoiceTime=TempHint['time'][TempHint["hinttype"]=="voice"].sum()
    except:
        HintsVoiceTime=0

    try:  
        NumberOfWholeHint= TempHint["hinttype"].count()
    except:    
        NumberOfWholeHint=0
    
    try:
        NumberOfTextHint=TempHint["hinttype"].value_counts()["text"]
    except KeyError as er:
        NumberOfTextHint=0
    
    try:    
        NumberOfImageHint=TempHint["hinttype"].value_counts()["image"]
    except KeyError as er:
        NumberOfImageHint=0   
    
    try:    
        NumberOfVoiceHint=TempHint["hinttype"].value_counts()["voice"]
    except KeyError as er:
        NumberOfVoiceHint=0     

    try:    
        RevisitHint = TempHint[TempHint.duplicated(["hintname", "hinttype"])]    
        NumberOfWholHintRevisit = TempHint.duplicated(["hintname", "hinttype"]).sum()
    except:
        NumberOfWholHintRevisit=0
    
    try:    
        NumberOfTextHintRevisit=RevisitHint["hinttype"].value_counts()["text"]
    except KeyError as er:
        NumberOfTextHintRevisit=0  

    try:    
        NumberOfImageHintRevisit=RevisitHint["hinttype"].value_counts()["image"]
    except KeyError as er:
        NumberOfImageHintRevisit=0 
    try:    
        NumberOfVoiceHintRevisit=RevisitHint["hinttype"].value_counts()["voice"]
    except KeyError as er:
        NumberOfVoiceHintRevisit=0 
        
    return (TotalHintTime, HintsTextTime, HintsImageTime, HintsVoiceTime, NumberOfWholeHint, NumberOfTextHint, NumberOfImageHint, 
            NumberOfVoiceHint, NumberOfWholHintRevisit, NumberOfTextHintRevisit, NumberOfImageHintRevisit, NumberOfVoiceHintRevisit)    
    
         
    

In [3]:
#Extract Features from Words File

def Words_Features(email):
    TempWord=Words[(Words["userid"] == email)]
    TotalWordsTime = TempWord['time'].sum()
    TimesOnQuestions=TempWord['time'][TempWord["id"].str.contains("q")].sum()
    TimesOnText=TotalWordsTime-TimesOnQuestions

    NumberofWholeMouseMovement=TempWord["id"].count()
    NumberOfMouseRevisit=TempWord.duplicated(["id","slide"]).sum()
    RevisitPercentage=NumberOfMouseRevisit/NumberofWholeMouseMovement

    WholeRevisitWords = TempWord[TempWord.duplicated(["id", "slide"])].drop_duplicates(subset=["id", "slide"])
    NumberOfWholeWordsRevisit=WholeRevisitWords["id"].count()
    NumberOfQuestionWordsRevisit=WholeRevisitWords["id"].str.contains("q").sum()
    NumberOfTextWordsRevisit=NumberOfWholeWordsRevisit-NumberOfQuestionWordsRevisit
    
    return(TotalWordsTime, TimesOnQuestions, TimesOnText, NumberofWholeMouseMovement, NumberOfMouseRevisit, RevisitPercentage, NumberOfWholeWordsRevisit, NumberOfQuestionWordsRevisit, NumberOfTextWordsRevisit)
   

In [4]:

def Words_Transition(email):
    global word
    global prevoiusword
    TempWord=Words[(Words["userid"] == Email)]
    TempWord=TempWord.reset_index()
    Count=0
    for index, row in TempWord.iterrows():
        if index!=0:
            word=TempWord["id"][index]
            prevoiusword=TempWord["id"][index-1]
            if "q" in word and "q" not in prevoiusword:
                Count=Count+1
            if  "q" not in word and "q"  in prevoiusword:
                Count=Count+1
    return(Count)            
    
    

In [5]:
def Hints_Transition(email):
    global word
    global prevoiusword
    TempHint=Hints[(Hints["userid"] == email)] 
    TempHint=TempHint.reset_index()
    Count=0;
    for index, row in TempHint.iterrows():
        if index!=0:
            word=TempHint["hinttype"][index]
            prevoiusword=TempHint["hinttype"][index-1]
            if word!=prevoiusword:
                Count=Count+1
    return(Count)


In [6]:
def Execute_Performance(email):
    TempQuestion=Questions[(Questions["userid"] == email)] 
    TempQuestion=TempQuestion.reset_index()
    array = np.array([[1,3,1],[2,3,1],[3,4,1],[4,4,1],[1,3,2],[2,2,2],[3,4,2],[4,1,2],[1,4,3],[2,4,3],[3,1,3],[4,3,3]])
    answers = pd.DataFrame(array, columns=['questionnumber','questionanswer','slide'])
    questions=pd.DataFrame(TempQuestion, columns =['questionnumber','questionanswer','slide'])
    TrueAnswers=answers.eq(questions).all(axis=1).sum()
    Performance=TrueAnswers/12*100
    return(Performance)

In [7]:
#import Features to Csv File

def Import_to_csvfile():
    
    List=[TotalHintTime,HintsTextTime,HintsImageTime,HintsVoiceTime,NumberOfWholeHint,NumberOfTextHint,NumberOfImageHint,NumberOfVoiceHint,NumberOfWholHintRevisit,NumberOfTextHintRevisit,NumberOfImageHintRevisit,NumberOfVoiceHintRevisit,
    TotalWordsTime,TimesOnQuestions,TimesOnText,NumberofWholeMouseMovement,NumberOfMouseRevisit,RevisitPercentage,NumberOfWholeWordsRevisit,NumberOfQuestionWordsRevisit,NumberOfTextWordsRevisit,
    NumberOfTransitionBetweenTextandQuestion,NumberOfTransitionBetweenHints,Performance]
    from openpyxl import load_workbook
    with open('Outpu.csv', 'a') as f_object:
        
        writer_object = writer(f_object)
        writer_object.writerow(List)
        f_object.close()

In [8]:
for i in range(NumberOfUser):
    Email=Users["_id"][i]
    TotalHintTime, HintsTextTime, HintsImageTime, HintsVoiceTime, NumberOfWholeHint, NumberOfTextHint, NumberOfImageHint, NumberOfVoiceHint, NumberOfWholHintRevisit, NumberOfTextHintRevisit, NumberOfImageHintRevisit, NumberOfVoiceHintRevisit  = Hints_Features(Email)
    TotalWordsTime, TimesOnQuestions, TimesOnText, NumberofWholeMouseMovement, NumberOfMouseRevisit, RevisitPercentage, NumberOfWholeWordsRevisit, NumberOfQuestionWordsRevisit, NumberOfTextWordsRevisit = Words_Features(Email)
    NumberOfTransitionBetweenTextandQuestion = Words_Transition(Email)
    NumberOfTransitionBetweenHints=Hints_Transition(Email)
    Performance=Execute_Performance(Email)
    Import_to_csvfile()
    